In [28]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
from dask.distributed import Client, LocalCluster, wait
from matplotlib.animation import FuncAnimation

In [29]:
plt.style.use('dark_background')

In [30]:
cluster = LocalCluster(n_workers=8, threads_per_worker=2, memory_limit=32e9,
                       silence_logs=50)
client = Client(address=cluster)

/gxfs_home/geomar/smomw352/miniconda3/envs/py3_std/lib/python3.7/site-packages/distributed/node.py:164: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41391 instead
  expected, actual


In [31]:
client

Client Scheduler: tcp://127.0.0.1:42463 Dashboard: http://127.0.0.1:41391/status,Cluster Workers: 8 Cores: 16 Memory: 238.42 GiB


[PosixPath('/gxfs_work1/geomar/smomw352/mask_orca12.nc'), PosixPath('/gxfs_work1/geomar/smomw352/mesh_hgr_orca12.nc'), PosixPath('/gxfs_work1/geomar/smomw352/mesh_zgr_orca12.nc')]


## Read data

In [32]:
expnames = ['FOCI_BJK019']
times = [slice('1953-01-01','1956-12-31')]

In [33]:
t_5d_all = []
u_5d_all = []
v_5d_all = []
i_5d_all = []

for i, (exp,time) in enumerate( zip(expnames, times) ):
    
    time_str = '*'
    
    chunks = {'x':4322, 'y':3059, 'deptht': 1, 'depthu':1, 'depthv':1}
    
    ddir = '/gxfs_work1/geomar/smomw352/esm-experiments/%s/outdata/nemo/' % (exp,)
    tfiles = '%s/%s_5d_%s_grid_T.nc' % (ddir,exp,time_str)
    print(tfiles)
    t_5d = xr.open_mfdataset(tfiles,combine='by_coords', chunks=chunks).rename({'time_counter':'time'}).sel(time=time)
    
    ufiles = '%s/%s_5d_%s_grid_U.nc' % (ddir,exp,time_str)
    u_5d = xr.open_mfdataset(ufiles,combine='by_coords', chunks=chunks).rename({'time_counter':'time'}).sel(time=time)
    
    vfiles = '%s/%s_5d_%s_grid_V.nc' % (ddir,exp,time_str)
    v_5d = xr.open_mfdataset(vfiles,combine='by_coords', chunks=chunks).rename({'time_counter':'time'}).sel(time=time)
    
    ifiles = '%s/%s_5d_%s_icemod.nc' % (ddir,exp,time_str)
    i_5d = xr.open_mfdataset(ifiles,combine='by_coords', chunks=chunks).rename({'time_counter':'time'}).sel(time=time)

    t_5d_all.append(t_5d)
    u_5d_all.append(u_5d)
    v_5d_all.append(v_5d)
    i_5d_all.append(i_5d)

/gxfs_work1/geomar/smomw352/esm-experiments/FOCI_BJK019/outdata/nemo//FOCI_BJK019_5d_*_grid_T.nc


In [34]:
t_5d

<xarray.Dataset>
Dimensions:               (time: 284, y: 3059, x: 4322, deptht: 75, axis_nbounds: 2)
Coordinates:
    nav_lat               (time, y, x) float32 dask.array<chunksize=(6, 2364, 2161), meta=np.ndarray>
    nav_lon               (time, y, x) float32 dask.array<chunksize=(6, 2364, 2161), meta=np.ndarray>
  * deptht                (deptht) float32 0.5058 1.556 ... 5.698e+03 5.902e+03
    time_centered         (time) datetime64[ns] dask.array<chunksize=(6,), meta=np.ndarray>
  * time                  (time) datetime64[ns] 1953-01-03T12:00:00 ... 1956-...
Dimensions without coordinates: y, x, axis_nbounds
Data variables: (12/18)
    deptht_bounds         (time, deptht, axis_nbounds) float32 dask.array<chunksize=(6, 1, 2), meta=np.ndarray>
    time_centered_bounds  (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(6, 2), meta=np.ndarray>
    time_counter_bounds   (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(6, 2), meta=np.ndarray>
    toce                  (time, deptht, y, x) float32 dask.array<chunksize=(6, 1, 3059, 4322), meta=np.ndarray>
    soce                  (time, deptht, y, x) float32 dask.array<chunksize=(6, 1, 3059, 4322), meta=np.ndarray>
    mldr10_1              (time, y, x) float32 dask.array<chunksize=(6, 3059, 4322), meta=np.ndarray>
    ...                    ...
    qsr                   (time, y, x) float32 dask.array<chunksize=(6, 3059, 4322), meta=np.ndarray>
    qt                    (time, y, x) float32 dask.array<chunksize=(6, 3059, 4322), meta=np.ndarray>
    qns_cal               (time, y, x) float32 dask.array<chunksize=(6, 3059, 4322), meta=np.ndarray>
    sst                   (time, y, x) float32 dask.array<chunksize=(6, 3059, 4322), meta=np.ndarray>
    sss                   (time, y, x) float32 dask.array<chunksize=(6, 3059, 4322), meta=np.ndarray>
    ssh                   (time, y, x) float32 dask.array<chunksize=(6, 3059, 4322), meta=np.ndarray>
Attributes:
    name:         FOCI_BJK019_5d_19530101_19530131_grid_T
    description:  ocean T grid variables
    title:        ocean T grid variables
    Conventions:  CF-1.6
    timeStamp:    2022-Aug-23 19:03:38 GMT
    uuid:         f21abaf7-9b6d-40ff-9f59-7e7156c178a4
    history:      Thu Sep  1 12:02:15 2022: ncks -O -7 --no-alphabetize -L 1 ...
    NCO:          netCDF Operators version 4.9.1 (Homepage = http://nco.sf.ne...

In [44]:
ds_t = xr.concat(t_5d_all, dim='time').chunk({'time':1})
ds_u = xr.concat(u_5d_all, dim='time').chunk({'time':1})
ds_v = xr.concat(v_5d_all, dim='time').chunk({'time':1})
ds_i = xr.concat(i_5d_all, dim='time').chunk({'time':1})

In [45]:
ds_u

<xarray.Dataset>
Dimensions:               (time: 284, y: 3059, x: 4322, depthu: 75, axis_nbounds: 2)
Coordinates:
    nav_lat               (time, y, x) float32 dask.array<chunksize=(1, 2364, 2161), meta=np.ndarray>
    nav_lon               (time, y, x) float32 dask.array<chunksize=(1, 2364, 2161), meta=np.ndarray>
  * depthu                (depthu) float32 0.5058 1.556 ... 5.698e+03 5.902e+03
    time_centered         (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * time                  (time) datetime64[ns] 1953-01-03T12:00:00 ... 1956-...
Dimensions without coordinates: y, x, axis_nbounds
Data variables:
    depthu_bounds         (time, depthu, axis_nbounds) float32 dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    time_centered_bounds  (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds   (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    uoce                  (time, depthu, y, x) float32 dask.array<chunksize=(1, 1, 3059, 4322), meta=np.ndarray>
    utau                  (time, y, x) float32 dask.array<chunksize=(1, 3059, 4322), meta=np.ndarray>
Attributes:
    name:         FOCI_BJK019_5d_19530101_19530131_grid_U
    description:  ocean U grid variables
    title:        ocean U grid variables
    Conventions:  CF-1.6
    timeStamp:    2022-Aug-23 19:03:41 GMT
    uuid:         65a0a56b-a9fc-4fa6-8299-e555c7d4eab1
    history:      Thu Sep  1 12:29:07 2022: ncks -O -7 --no-alphabetize -L 1 ...
    NCO:          netCDF Operators version 4.9.1 (Homepage = http://nco.sf.ne...

## Calculate surface speed

In [47]:
ds_u['uoce'].rename({'depthu':'deptht'}).isel(deptht=0)

<xarray.DataArray 'uoce' (time: 284, y: 3059, x: 4322)>
dask.array<getitem, shape=(284, 3059, 4322), dtype=float32, chunksize=(1, 3059, 4322), chunktype=numpy.ndarray>
Coordinates:
    nav_lat        (time, y, x) float32 dask.array<chunksize=(1, 2364, 2161), meta=np.ndarray>
    nav_lon        (time, y, x) float32 dask.array<chunksize=(1, 2364, 2161), meta=np.ndarray>
    deptht         float32 0.5058
    time_centered  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * time           (time) datetime64[ns] 1953-01-03T12:00:00 ... 1956-12-28T1...
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_x_velocity
    long_name:           ocean current along i-axis
    units:               m/s
    online_operation:    average
    interval_operation:  300 s
    interval_write:      5 d
    cell_methods:        time: mean (interval: 300 s)

In [48]:
da_speed = ((ds_u['uoce'].rename({'depthu':'deptht'}).isel(deptht=0)**2 + 
             ds_v['voce'].rename({'depthv':'deptht'}).isel(deptht=0)**2)**0.5).chunk({'time':1})#.compute()

In [50]:
da_speed

<xarray.DataArray (time: 284, y: 3059, x: 4322)>
dask.array<pow, shape=(284, 3059, 4322), dtype=float32, chunksize=(1, 3059, 4322), chunktype=numpy.ndarray>
Coordinates:
    deptht         float32 0.5058
    time_centered  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * time           (time) datetime64[ns] 1953-01-03T12:00:00 ... 1956-12-28T1...
Dimensions without coordinates: y, x

## Plot speeds

In [60]:
def plot_speed(speed, region='global', save=False):
    
    fig1, ax1 = plt.subplots(1,1, figsize=(10,6))
    
    # this will make NaNs black
    ax1.set_facecolor('black')
    
    _v = speed
    _vm = _v.where(_v != 0)
    
    cbar_kwargs = {'orientation':'vertical', 'pad':0.02, 'label':'[m/s]'}
    _vm.plot.pcolormesh(ax=ax1, cmap=cmocean.cm.speed, 
                        vmin=0, vmax=1, 
                        cbar_kwargs=cbar_kwargs)
    
    ax1.set_xticks([])
    ax1.set_xticklabels([])
    ax1.set_yticks([])
    ax1.set_yticklabels([])
    ax1.set_xlabel('')
    ax1.set_ylabel('')
    ax1.set_title('Surface speed')
    
    if save:
        fig1.savefig('frames/surface_speed_%s_%04d.png' % (region,ji), format='png', dpi=300)
        plt.close(fig1)
        

In [ ]:
nsteps = len(da_speed['time'])

for ji in range(0, nsteps):
    
    _speed = da_speed.isel(time=ji)
    
    plot_speed(_speed, save=True)
    

## Plot SST and ice

In [11]:
# Number of time steps
nsteps = len(ds_t['time'])
nsteps

284

In [26]:
def plot_sst_ice(sst, ice, region='global', save=False):
    
    fig1 = plt.figure(figsize=(8,6))
    
    ax1 = fig1.add_axes([0.1,0.1,0.8,0.8])
    cax1 = fig1.add_axes([0.92, 0.1, 0.01, 0.39])
    cax2 = fig1.add_axes([0.92, 0.51, 0.01, 0.39])
    
    # this will make NaNs black
    ax1.set_facecolor('Gainsboro')
    
    # apply mask
    _sst = sst.where(sst != 0)
    _ice = ice.where(ice > 0.15) * 100
    
    if region == 'weddell':
        vmin, vmax = -2, 30
    else:
        vmin, vmax = -2, 30
    
    cbar_kwargs = {'orientation':'vertical', 'pad':0.02, 'label':'[C]'}
    _sst.plot.pcolormesh(ax=ax1, cmap=cmocean.cm.thermal, 
                         vmin=vmin, vmax=vmax, 
                         cbar_kwargs=cbar_kwargs,
                         cbar_ax=cax1,
                         rasterized=True)
    
    cbar_kwargs = {'orientation':'vertical', 'pad':0.02, 'label':'[%]'}
    _ice.plot.pcolormesh(ax=ax1, cmap=cmocean.cm.ice, 
                         vmin=0, vmax=100, 
                         cbar_kwargs=cbar_kwargs,
                         cbar_ax=cax2,
                         rasterized=True)
    
    if region == 'weddell':
        ax1.set_xlim([2517, 3897])
        ax1.set_ylim([61,   1111])
    
    ax1.set_xticks([])
    ax1.set_xlabel('')
    ax1.set_xticklabels([])
    ax1.set_yticks([])
    ax1.set_yticklabels([])
    ax1.set_ylabel('')
    ax1.set_title('SST and sea-ice concentration')
    
    if save:
        fig1.savefig('frames/sst_ice_%s_%04d.png' % (region,ji), format='png', dpi=300)
        plt.close(fig1)
    

In [18]:
for ji in range(0, nsteps):
    
    # load in the data
    sst = ds_t['sst'].isel(time=ji).load()
    ice = ds_i['ileadfra'].isel(time=ji).load()
    
    fig1 = plt.figure(figsize=(8,6))
    
    ax1 = fig1.add_axes([0.1,0.1,0.8,0.8])
    cax1 = fig1.add_axes([0.92, 0.1, 0.01, 0.39])
    cax2 = fig1.add_axes([0.92, 0.51, 0.01, 0.39])
    
    # this will make NaNs black
    ax1.set_facecolor('Gainsboro')
    
    # apply mask
    _sst = sst.where(sst != 0)
    _ice = ice.where(ice > 0.15) * 100
    
    cbar_kwargs = {'orientation':'vertical', 'pad':0.02, 'label':'[C]'}
    _sst.plot.pcolormesh(ax=ax1, cmap=cmocean.cm.thermal, 
                         vmin=-2, vmax=30, 
                         cbar_kwargs=cbar_kwargs,
                         cbar_ax=cax1,
                         rasterized=True)
    
    cbar_kwargs = {'orientation':'vertical', 'pad':0.02, 'label':'[%]'}
    _ice.plot.pcolormesh(ax=ax1, cmap=cmocean.cm.ice, 
                         vmin=0, vmax=100, 
                         cbar_kwargs=cbar_kwargs,
                         cbar_ax=cax2,
                         rasterized=True)
    
    ax1.set_xticks([])
    ax1.set_xlabel('')
    ax1.set_xticklabels([])
    ax1.set_yticks([])
    ax1.set_yticklabels([])
    ax1.set_ylabel('')
    ax1.set_title('SST and sea-ice concentration')
    
    fig1.savefig('frames/sst_ice_%04d.png' % (ji,), format='png', dpi=300)
    plt.close(fig1)

In [27]:
for ji in range(0, nsteps):
    
    # load in the data
    sst = ds_t['sst'].isel(time=ji).load()
    ice = ds_i['ileadfra'].isel(time=ji).load()
    
    plot_sst_ice(sst, ice, region='weddell', save=True)